In [5]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 3.7 MB/s eta 0:00:00
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 17.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.4/392.4 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 45.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.9 MB/s eta 0:00:00
Using cached tenacity-8.2.3-py3-none-an

In [11]:
from langchain.llms import OpenAI
import os 

In [116]:
os.environ["OPEN_AI_KEY"] = "XXXXXXXXX"

Temperature is just like creativity with each new command.

In [21]:
llm = OpenAI(openai_api_key = os.environ["OPEN_AI_KEY"], temperature=0.6)

In [22]:
text = "how is pm of india"
print(llm.predict(text))

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

#### Using Hugging Face for text-to-text generation

In [82]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "XXXXXXX"

In [83]:
from langchain import HuggingFaceHub
llm_flan_large = HuggingFaceHub(repo_id = "google/flan-t5-large", model_kwargs = {"temperature" : 0.6, "max_length" : 100})

In [85]:
llm_flan_large.predict("write a poem on the ai")

'i love the ai i love the ai i love the ai i love the ai i love the ai i love the ai i love the ai i love the ai i love the ai i love the ai i love the ai i love the ai i love the ai i love the ai '

#### Prompt Templates

In [86]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate(input_variables=['country'], 
                                 template= "Tell me the capital of this {country}")
prompt_template.format(country = "Pakistan")



'Tell me the capital of this Pakistan'

In [88]:
from langchain.chains import LLMChain


In [90]:
llm_chain = LLMChain(llm = llm_flan_large, prompt = prompt_template)
print(llm_chain.run("Pakistan"))

islamabad


### Multiple Chaining

In [93]:
#to get the country capital
country_capital = PromptTemplate(input_variables = ['country'],
                                 template= "What is the capital of {country}")
capital_chain = LLMChain(llm = llm_flan_large, prompt = country_capital)

#to get the more details about the country
famouns_places = PromptTemplate(input_variables= ['capital'],
                                template="What are the famous places to visit in {capital}")
famous_chain = LLMChain(llm = llm_flan_large, prompt = famouns_places)


In [97]:
from langchain.chains import SimpleSequentialChain

In [101]:
simples_equential_chain = SimpleSequentialChain(chains =[capital_chain, famous_chain])
simples_equential_chain.run("Pakistan")

'The National Museum of Pakistan'

In [102]:
## Get the specific output

#to get the country capital
country_capital = PromptTemplate(input_variables = ['country'],
                                 template= "What is the capital of {country}")
capital_chain = LLMChain(llm = llm_flan_large, prompt = country_capital, output_key= "capital")

#to get the more details about the country
famouns_places = PromptTemplate(input_variables= ['capital'],
                                template="What are the famous places to visit in {capital}")
famous_chain = LLMChain(llm = llm_flan_large, prompt = famouns_places, output_key= "places")

In [103]:
from langchain.chains import SequentialChain
seq_chain = SequentialChain(chains = [capital_chain, famous_chain],
                input_variables = ['country'],
                output_variables = ['capital', 'places'])


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'country': 'Pakistan',
 'capital': 'islamabad',
 'places': 'The National Museum of Pakistan'}

In [110]:
output = seq_chain({"country": "Pakistan"})
print("The capital of given country is: ",output['capital'])
print("Lastly the famous places to visit is: ",output['places'])


The capital of given country is:  islamabad
Lastly the famous places to visit is:  The National Museum of Pakistan


### Chatmodels with ChatOpenAI

In [112]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage

In [117]:
chat_llm = ChatOpenAI(openai_api_key = os.environ["OPENAI_API_KEY"], temperature= 0.6, model= "gpt-3.5-turbo")

In [118]:
chat_llm([
    SystemMessage(content="You are a swimming teacher"),
    HumanMessage(content= "What is swimming")
])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### Prompt Template + LLM +Output Parsers

In [119]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [120]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [121]:
template="Your are a helpful assistant. When the use given any input , you should generate 5 words synonyms in a comma seperated list"
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)


])

In [125]:
chain=chatprompt|llm_flan_large|Commaseperatedoutput()

In [127]:
chain.invoke({"text":"intelligent"})

['intelligent human']